# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### Структура проекта

1. Загрузка данных
2. Умножение матриц
3. Алгоритм преобразования
4. Проверка алгоритма
5. Итоговый вывод

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# загружаем данные
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
# выводим общую информацию по таблце и первые 5 строк
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


* В таблице 5000 строк и 5 столбцов:
    * 4 столбца - признаки;
    * 1 столбец `'Страховые выплаты'` - целевой.
* Пропущенные значения отсутствуют. Типы данных соответствуют хранящейся информации.
* Проверим наличие дубликатов.

In [4]:
print(f'Количество дубликатов: {df.duplicated().sum()}')

Количество дубликатов: 153


Обнаружено 153 дублирующие строки, удалим их.

In [5]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
print(f'Осталось строк: {df.shape[0]}')

Осталось строк: 4847


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:**

Допустим, 

$$
X_1 = X P
$$

Вычислим, чему будет равено предсказание и вектор весов.

$$
w_1 = ((X_1)^T X_1)^{-1}(X_1)^T y = ((XP)^T XP)^{-1}(XP)^T y
$$

Используем свойство транспорнированной матрицы и раскроем внутренние скобки:

$$
w_1= (P^T X^T XP)^{-1} P^T X^T y
$$

Используем свойсво обратной матрицы и вынесем за скобки $P$ и $P^T$:

$$
w_1= P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Подставим получившееся выражение вектора $w_1$ в формулу предсказания $a_1 = XP w_1$.

$$
a_1 = XP P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$


Умножение приведет к $P P^{-1} = E$ и $(P^T)^{-1} P^T = E$. Умножение на единичную матрицу ничего не меняет. Тогда,

$$
a_1 = X(X^T X)^{-1} X^T y = X w = a
$$


Как видно, значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу.

## 3. Алгоритм преобразования

**Алгоритм**

Создадим случайную обратимую матрицу $P$, затем умножим матрицу с признаками $X$ на матрицу $P$.

$$
X * P = Z
$$

Для возврата к исходной матрице $X$, умножим новую матрицу $Z$ на обратную матрицу $P^{-1}$.

$$
Z * P^{-1} = X
$$

**Обоснование**

Как было доказано в шаге 2, качество линейной регрессии не изменится при умножении матрицы признаков на любую обратимую матрицу P. Поэтому можно следовать описанному выше алгоритму без опасения получить другой результат линейной регрессии.

## 4. Проверка алгоритма

* Обучим модель линейной регрессии на исходных данных и высчитаем метрику R2.
* Создадим случайную матрицу нужного размера с помощью инструмента `random.normal` библиотеки `numpy` и преобразуем данные по вышеопределенному алгоритму.
* Обучим модель линейной регрессии на преобразованных данных и высчитаем метрику R2.
* Сравним метрики для двух моделей.

In [6]:
# делим выборку на целевой и остальные признаки
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [7]:
# делим датасет на обучающую и тестовую выборки
X, X_test, y, y_test = train_test_split(features, target, test_size=0.25, random_state=1505)

In [8]:
# инициируем модель линейной регрессии
model = LinearRegression()
model.fit(X, y)
r2_origin_data = r2_score(y_test, model.predict(X_test))
print(f'Значение метрики R2 на исходных данных: {r2_origin_data}')

Значение метрики R2 на исходных данных: 0.44480062344521076


In [9]:
# генерируем случайную матрицу необходимого размера (в соответствии с количеством признаков 4)
P = np.random.normal(size=(4, 4))
# проверяем матрицу на обратимость, при использовании необратимой матрицы дальнейшая работа по описанному алгоритму невозможна
# если матрица необратима, код выдаст ошибку
np.linalg.inv(P)

array([[ 0.57319379, -0.81825878,  0.66521545,  0.31481766],
       [-0.28533325,  0.18761085, -0.94809787, -0.04025196],
       [-1.45305087,  0.51873984, -0.10725015,  0.63848398],
       [ 1.38141764, -0.6838156 ,  0.55524771,  0.56307345]])

In [10]:
# преобразовываем исходные данные по алгоритму
features_new = features.values @ P

# делим полученные данные на обучающую и тестовую выборки
X_new, X_new_test = train_test_split(features_new, test_size=0.25, random_state=1505)

In [11]:
# инициируем модель линейной регрессии
model = LinearRegression()
model.fit(X_new, y)
r2_new_data = r2_score(y_test, model.predict(X_new_test))
print(f'Значение метрики R2 на преобразованных данных: {r2_new_data}')

Значение метрики R2 на преобразованных данных: 0.44480062344521054


Сравним показатели качества моделей.

In [12]:
result = pd.DataFrame(data= [r2_origin_data,
                      r2_new_data], 
                     columns=['R2'], 
                     index=['Линейная регрессия на исходных данных',
                            'Линейная регрессия на преобразованных признаках',])
result

,R2
Линейная регрессия на исходных данных,0.444801
Линейная регрессия на преобразованных признаках,0.444801


## Итоговый вывод

1. Данные были загружены и изучены. Пропуски отсутствуют, типы данных соответствуют хранимой информации. Найдены и устранены дубликаты.
2. Теоретически доказано, что качество линейной регресии не меняется от использования исходной матрицы, умноженной на обратимую.
3. Создан алгоритм преобразования данных.
4. Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с ним:
    * Метрика `R2` для Линейной регрессии на исходных данных **0.444801**
    * Метрика `R2` для Линейной регрессии на преобразованных признаках **0.444801**